## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham". 

### Данные

Для удобства данные приложены в описании задания

In [19]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

*Оставляем* только интересующие нас колонки — тексты смс и метки:

In [20]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [21]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [22]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [23]:
from nltk import stem
from nltk.corpus import stopwords
import nltk
import re

nltk.download('stopwords')
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess1(text):
    text = re.sub(r'[^\w\s]', '', text)
    # Ваш код здесь
    return text


def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    # Stem words
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверка, что функция работает верно

In [24]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [25]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [26]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=1)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [29]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 51, C = 1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [30]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.996     0.986      1128
           1      0.965     0.836     0.896       165

    accuracy                          0.975      1293
   macro avg      0.971     0.916     0.941      1293
weighted avg      0.975     0.975     0.974      1293



Выполним предсказание для конкретного текста

In [31]:
txt = "As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a å£1500 Bonus Prize, call 09066364589"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [32]:
model.predict(txt)

array([1])

In [ ]:
Сообщение помечено как спам.

In [33]:
from sklearn.metrics import precision_recall_fscore_support

# Evaluate model on test data
predictions = model.predict(X_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, predictions, average='macro')

# Print precision, recall and F1-score (macro avg)
print(f"Precision (macro avg): {precision:.4f}")
print(f"Recall (macro avg): {recall:.4f}")
print(f"F1-score (macro avg): {f1_score:.4f}")

# Make a prediction for a given text
str_list = ['Hello, I have a special offer for you… WANT TO LOSE WEIGHT? The most powerful weightloss is now available without prescription. All natural Adipren720 100% Money Back Guarantee!',
            'So the summer has flown by - today a new school year begins. Ahead - new students and pupils; knowledge, discoveries and accomplishments, checking abstracts and coursework, conducting colloquiums and exams.',
            "Poorly written I'm giving up on this book.",
            'Screw this self-help industry crap.',
            ]
txt = "As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a å£1500 Bonus Prize, call 09066364589"
for txt in str_list:
  txt = preprocess(txt)
  txt = vectorizer.transform([txt])
  prediction = model.predict(txt)
  print("Prediction:", "spam" if prediction[0] == 1 else "ham")

Precision (macro avg): 0.9708
Recall (macro avg): 0.9160
F1-score (macro avg): 0.9410
Prediction: ham
Prediction: ham
Prediction: ham
Prediction: ham
